### Stuff

In [3]:
from dotenv import dotenv_values
import os

env_vars = dotenv_values('../.env')
os.environ['OPENAI_API_KEY'] = env_vars.get('OPENAI_API_KEY')

In [9]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.chains import RetrievalQA
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough

In [5]:
model = ChatOpenAI(temperature=0.1)
cache_dir = LocalFileStore("./.cache/")
splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator=".",
    chunk_size=600,
    chunk_overlap=100,
)
loader = UnstructuredFileLoader("./file/law.txt")
docs = loader.load_and_split(text_splitter=splitter)
embeddings = OpenAIEmbeddings()
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)
vectorstore = FAISS.from_documents(docs, cached_embeddings)

In [11]:
retriver = vectorstore.as_retriever()

In [13]:
retriver

VectorStoreRetriever(tags=['FAISS', 'CacheBackedEmbeddings'], vectorstore=<langchain.vectorstores.faiss.FAISS object at 0x7f198a73f350>)

In [ ]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "너는 AI 어시스턴스입니다. 주어진 정보만을 이용하여 질문에 대답하세요. 답을 모르면 모른다고 하세요. 모르는 정보를 지어내지 마세요. don't make it up:\n\n{context}"),
    ("human", "{question}")
])
chain = {"context": retriver, "question": RunnablePassthrough()} | prompt | model

In [12]:
chain.invoke("제1조는 무엇입니까?")

AIMessage(content='제1조는 "대한민국은 민주공화국이다. 한민국의 주권은 국민에게 있고, 모든 권력은 국민으로부터 나온다."라는 내용을 담고 있습니다.')

### Map Reduce